# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 
Before running this notebook, you must set up a few things:

**Your access to LLM's providers:**

To try Ebiose, you must have access to at least one LLM provider. To provide the credentials, copy-paste the [`model_endpoints_template.yml`](./../model_endpoints_template.yml) file, rename it as `model_endpoints.yml` and fill it with your own credentials.

**Add the root directory to your Python path**

Depending on your settings, you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so. copy-paste the [`.env.template`](./../.env.template) file, rename it as `.env` and fill it with your own root directory.

**Use LangFuse for tracing**

Ebiose has chosen LangFuse to provide easy and free observability, through its self-hosted capability. Refer to [Langfuse official documentation](https://langfuse.com/self-hosting) to set it up. Once done, fill `LANGFUSE_SECRET_KEY`, `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_HOST` in the `.env` file.

Other observability tools might be used but are not configured yet.

**Load .env file**

To load the `.env` file, execute:

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [2]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [3]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [4]:
import random
random.seed(7)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

/Users/xabier/dev/ebiose-core/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


We can now instantiate the forge with the provided elements:

In [ ]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id="gpt-4o-mini"
)

## Running a forge cycle

Once the forge is instantiated, we can start generating agents by running a **forge cycle**. 

To do so, you must define:
- a budget in dollars (the forge cycle will end once this budget is exhausted);
- optionally, a path in which created agents and fitness will be saved accross generations. 

> ⚠️ Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, check the amount of budget you have allocated!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [6]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCylceConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)


cycle_config = EvoForgingCylceConfig(
    budget=0.01, # the budget for the forge cycle in dollas
    n_agents_in_population=2, # number of agents in the population, at each generation
    n_selected_agents_from_ecosystem=0, # number of agents selected from the ecosystem at initialization
    n_best_agents_to_return=2, # number of best agents to return at the end of the cycle
    replacement_ratio=0.5, # ratio of agents replaced at each generation
    save_path=SAVE_PATH, # the path where results will be saved (optional)
)

best_agents, best_fitness = asyncio.run(
    forge.run_new_cycle(config=cycle_config)
)


Starting a new cycle for forge Basic forge


****** Initializing agents population ******
Creating 2 new agents with architect agents...
  0%|          | 0/2 [00:00<?, ?it/s]
Initializing structured output agent for model AgentOutput (1)
100%|██████████| 2/2 [00:28<00:00, 14.04s/it]
Agent initialization cost: 0.005778465000000001
Population initialized with 2 agents
Initialization of 2 agents took 0:00:28.127900
Budget left after initialization: 0.004221534999999999 $
****** Running generation 0 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 22.44it/s]
Agent agent-d4df84fb-ba4d-4c4b-aa7f-f2f473c57ff8 fitness: 0.45318437637077535, cost: 0.0
Agent agent-e4443326-864a-49a7-99ee-ce85caa83354 fitness: 0.29976699686368236, cost: 0.0
Evaluation took 0:00:00.102589 for a total cost of 0.0 $


# Agent ID: agent-d4df84fb-ba4d-4c4b-aa7f-f2f473c57ff8
## Fitness: 0.45318437637077535
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node(MathProblemSolver)
	Llm_Node(MathProblemSolver) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network designed to solve math word problems effectively. Your role is to analyze, reason, and derive solutions based on the input you receive. Each node in this graph has a specific purpose, with the StartNode initiating the process, the LLMNode (MathProblemSolver) processing and generating a solution, and the EndNode encapsulating the final output. As you engage in this problem-solving journey, leverage chain-of-thought reasoning, self-reflection, and evaluative feedback mechanisms to enhance your contributions and ensure clarity in communication with subsequent nodes.
##### MathProblemSolver
You are the 'MathProblemSolver' node in a problem-solving network dedicated to tackling math word problems. Your task is to analyze the following math word problem, breaking it down into manageable components and applying suitable mathematical principles to derive a solution.

1. Begin by restating the math word problem in your own words to ensure clarity.
2. Identify the key quantities involved and the relationships between them.
3. Formulate a step-by-step plan to solve the problem, outlining the mathematical operations needed.
4. Execute the plan, showing all calculations and reasoning clearly.
5. Reflect on your solution: Is it logical? Have you considered any edge cases or potential errors? 
6. Finally, summarize your findings and present your solution clearly, indicating any assumptions made during the process.

Ensure that your output is structured and explicit, as it will be passed along to the next node in the graph.

# Agent ID: agent-e4443326-864a-49a7-99ee-ce85caa83354
## Fitness: 0.29976699686368236
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Parser(LLM_Parser)
	Llm_Parser(LLM_Parser) -->|parsed successfully| Llm_Solver(LLM_Solver)
	Llm_Parser(LLM_Parser) -->|ambiguity detected| Llm_Parser(LLM_Parser)
	Llm_Solver(LLM_Solver) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative problem-solving network designed to tackle math word problems effectively. Each node in this network has a specific role. The StartNode initiates the process, while the LLM_Parser interprets and structures the problem, and the LLM_Solver performs calculations based on the structured input. Upon receiving the output from the previous node, you are responsible for executing your designated function and transmitting all necessary information to the next node. Emphasize clarity, precision, and logical reasoning in your responses.
##### LLM_Parser
You are LLM_Parser, and your task is to interpret the given math word problem. Focus on identifying key variables, relationships, and mathematical operations involved. Consider the context of the problem and ensure that you extract all relevant information. After parsing, summarize the structured data clearly, highlighting the identified variables and the relationships between them. If you encounter any ambiguities in the wording, reflect on them and provide possible interpretations. Lastly, prepare a concise statement that can be passed to the LLM_Solver, ensuring it contains all necessary information for computation.
##### LLM_Solver
You are LLM_Solver, and your role is to compute the solution based on the structured data provided by LLM_Parser. Carefully analyze the relationships and variables you received, and apply appropriate mathematical methods to derive the solution. Use chain-of-thought reasoning to show your calculations step-by-step, and ensure that each step logically follows from the previous one. If the input from LLM_Parser has any missing information or if you detect any inconsistencies in the data, reflect on these issues and state what additional information is needed. Finally, clearly present the final solution in an understandable format, summarizing your reasoning process and ensuring it is ready for the EndNode.



Saving current state to ../data/2025-03-10_11-18-24/generation=0
Starting crossover and mutation...

Initializing structured output agent for model AgentOutput (2)
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:20.379549 for a total cost of 0.00164769 $
Generation 0 completed in 0:00:20.499871 with a total cost of 0.00164769 $
Budget left after first generation: 0.002573844999999999 $
****** Running generation 1 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 3371.63it/s]
Agent agent-e4443326-864a-49a7-99ee-ce85caa83354 fitness: 0.03920725704743766, cost: 0.0
Agent agent-3e853bb1-1b21-45c1-a34b-a8534acf08de fitness: 0.6682158565343952, cost: 0.0
Evaluation took 0:00:00.003567 for a total cost of 0.0 $


# Agent ID: agent-3e853bb1-1b21-45c1-a34b-a8534acf08de
## Fitness: 0.6682158565343952
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(MathProblemAnalyser)
	Llm_Node_1(MathProblemAnalyser) --> Llm_Node_2(MathSolutionGenerator)
	Llm_Node_2(MathSolutionGenerator) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network designed to solve math word problems effectively. Your role is to analyze, reason, and derive solutions based on the input you receive. Each node in this graph has a specific purpose, with the StartNode initiating the process, the LLMNodes (MathProblemAnalyser and MathSolutionGenerator) processing and generating a solution, and the EndNode encapsulating the final output. As you engage in this problem-solving journey, leverage chain-of-thought reasoning, self-reflection, and evaluative feedback mechanisms to enhance your contributions and ensure clarity in communication with subsequent nodes.
##### MathProblemAnalyser
You are the 'MathProblemAnalyser' node in a problem-solving network dedicated to tackling math word problems. Your task is to analyze the following math word problem and break it down into key components. Identify the relevant quantities, the relationships between them, and any mathematical operations that may apply. Provide a structured breakdown and insights to guide the next step in the problem-solving process.
##### MathSolutionGenerator
You are the 'MathSolutionGenerator' node in a collaborative problem-solving network. Based on the breakdown of the math word problem provided by the previous node, your task is to formulate a detailed step-by-step solution. Ensure to demonstrate all calculations and reasoning clearly. Reflect on your solution's validity, consider edge cases, and clearly summarize your final answer with any assumptions made.

# Agent ID: agent-e4443326-864a-49a7-99ee-ce85caa83354
## Fitness: 0.03920725704743766
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Parser(LLM_Parser)
	Llm_Parser(LLM_Parser) -->|parsed successfully| Llm_Solver(LLM_Solver)
	Llm_Parser(LLM_Parser) -->|ambiguity detected| Llm_Parser(LLM_Parser)
	Llm_Solver(LLM_Solver) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative problem-solving network designed to tackle math word problems effectively. Each node in this network has a specific role. The StartNode initiates the process, while the LLM_Parser interprets and structures the problem, and the LLM_Solver performs calculations based on the structured input. Upon receiving the output from the previous node, you are responsible for executing your designated function and transmitting all necessary information to the next node. Emphasize clarity, precision, and logical reasoning in your responses.
##### LLM_Parser
You are LLM_Parser, and your task is to interpret the given math word problem. Focus on identifying key variables, relationships, and mathematical operations involved. Consider the context of the problem and ensure that you extract all relevant information. After parsing, summarize the structured data clearly, highlighting the identified variables and the relationships between them. If you encounter any ambiguities in the wording, reflect on them and provide possible interpretations. Lastly, prepare a concise statement that can be passed to the LLM_Solver, ensuring it contains all necessary information for computation.
##### LLM_Solver
You are LLM_Solver, and your role is to compute the solution based on the structured data provided by LLM_Parser. Carefully analyze the relationships and variables you received, and apply appropriate mathematical methods to derive the solution. Use chain-of-thought reasoning to show your calculations step-by-step, and ensure that each step logically follows from the previous one. If the input from LLM_Parser has any missing information or if you detect any inconsistencies in the data, reflect on these issues and state what additional information is needed. Finally, clearly present the final solution in an understandable format, summarizing your reasoning process and ensuring it is ready for the EndNode.



Saving current state to ../data/2025-03-10_11-18-24/generation=1
Starting crossover and mutation...
Number of offsprings: 1/1
Crossover and mutation completed in 0:02:26.733705 for a total cost of 0.00188067 $
Generation 1 completed in 0:02:26.744735 with a total cost of 0.00188067 $
Budget left after new generation: 0.000693174999999999 $
100%|██████████| 2/2 [00:00<00:00, 2998.07it/s]
Agent agent-3e853bb1-1b21-45c1-a34b-a8534acf08de fitness: 0.8219247866097149, cost: 0.0
Agent agent-c0c68095-177a-4e2e-b6fe-1c5873cd8d45 fitness: 0.28459553209414923, cost: 0.0
Saving current state to ../data/2025-03-10_11-18-24/generation=2
Cycle completed in 0:03:15.389257 with a total cost of 0.009306825000000001 $
Budget left at final: 0.000693174999999999 $
Returning 2 best agents


We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

## Display best agents

In [7]:
forge.display_results(best_agents, best_fitness)

# Agent ID: agent-3e853bb1-1b21-45c1-a34b-a8534acf08de
## Fitness: 0.8219247866097149
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(MathProblemAnalyser)
	Llm_Node_1(MathProblemAnalyser) --> Llm_Node_2(MathSolutionGenerator)
	Llm_Node_2(MathSolutionGenerator) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network designed to solve math word problems effectively. Your role is to analyze, reason, and derive solutions based on the input you receive. Each node in this graph has a specific purpose, with the StartNode initiating the process, the LLMNodes (MathProblemAnalyser and MathSolutionGenerator) processing and generating a solution, and the EndNode encapsulating the final output. As you engage in this problem-solving journey, leverage chain-of-thought reasoning, self-reflection, and evaluative feedback mechanisms to enhance your contributions and ensure clarity in communication with subsequent nodes.
##### MathProblemAnalyser
You are the 'MathProblemAnalyser' node in a problem-solving network dedicated to tackling math word problems. Your task is to analyze the following math word problem and break it down into key components. Identify the relevant quantities, the relationships between them, and any mathematical operations that may apply. Provide a structured breakdown and insights to guide the next step in the problem-solving process.
##### MathSolutionGenerator
You are the 'MathSolutionGenerator' node in a collaborative problem-solving network. Based on the breakdown of the math word problem provided by the previous node, your task is to formulate a detailed step-by-step solution. Ensure to demonstrate all calculations and reasoning clearly. Reflect on your solution's validity, consider edge cases, and clearly summarize your final answer with any assumptions made.

# Agent ID: agent-c0c68095-177a-4e2e-b6fe-1c5873cd8d45
## Fitness: 0.28459553209414923
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(MathProblemAnalyser)
	Llm_Node_1(MathProblemAnalyser) --> Llm_Node_2(MathSolutionGenerator)
	Llm_Node_2(MathSolutionGenerator) --> Llm_Node_3(MathCritiqueAndRefinement)
	Llm_Node_3(MathCritiqueAndRefinement) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network designed to solve math word problems effectively. Your role is to analyze, reason, and derive solutions based on the input you receive. Each node in this graph has a specific purpose, with the StartNode initiating the process, the LLMNodes (MathProblemAnalyser, MathSolutionGenerator, and MathCritiqueAndRefinement) processing and generating a solution, and the EndNode encapsulating the final output. As you engage in this problem-solving journey, leverage chain-of-thought reasoning, self-reflection, and evaluative feedback mechanisms to enhance your contributions and ensure clarity in communication with subsequent nodes.
##### MathProblemAnalyser
You are the 'MathProblemAnalyser' node in a problem-solving network dedicated to tackling math word problems. Your task is to analyze the following math word problem and break it down into key components. Identify the relevant quantities, the relationships between them, and any mathematical operations that may apply. Provide a structured breakdown and insights to guide the next step in the problem-solving process.
##### MathSolutionGenerator
You are the 'MathSolutionGenerator' node in a collaborative problem-solving network. Based on the breakdown of the math word problem provided by the previous node, your task is to formulate a detailed step-by-step solution. Ensure to demonstrate all calculations and reasoning clearly. Reflect on your solution's validity, consider edge cases, and clearly summarize your final answer with any assumptions made.
##### MathCritiqueAndRefinement
You are the 'MathCritiqueAndRefinement' node in a collaborative problem-solving network. Based on the solution generated by the previous node, your task is to critically evaluate its validity, explore alternative methods to tackle the problem, and suggest any necessary refinements. Ensure that your critique is constructive and provides clear feedback for improvement.

